# Gordon

In [38]:
import pandas as pd
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\markf\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [51]:
df_naama = pd.read_json("../data/experiment_naama.jsonl", lines=True)
df_naama 

,sentence,question,dependency,animacy,gender,distractor,correct_answer
0,The actress who was in the French director's s...,The actress hit the vase.,S-V,animate,נקבה,Match,true
1,The camera that was in the French director's s...,The camera hit the vase.,S-V,inanimate,נקבה,Mismatch,true
2,This is the actress who this week in the Frenc...,The actress hit the vase.,F-G,animate,נקבה,Match,true
3,This is the camera that this week in the Frenc...,The camera hit the vase.,F-G,inanimate,נקבה,Mismatch,true
4,The student who was yesterday in the class of ...,The student bored the class.,S-V,animate,נקבה,Match,true
...,...,...,...,...,...,...,...
155,This is the alarm that in the morning in the s...,The officer woke up the combat soldiers.,F-G,inanimate,נקבה,Mismatch,false
156,The witness who was this week in the hardened ...,The criminal indicted the accomplice.,S-V,animate,נקבה,Match,false
157,The evidence that was this week in the hardene...,The criminal indicted the accomplice.,S-V,inanimate,נקבה,Mismatch,false
158,This is the witness who this week in the harde...,The criminal indicted the accomplice.,F-G,animate,נקבה,Match,false


In [52]:
# add 'distractor' and 'label' columns

pd.options.mode.chained_assignment = None  # default='warn'

df_naama['match'] = df_naama['distractor'].apply(lambda x: True if x == 'Match' else False)

new_naama_df = df_naama[df_naama['correct_answer'] == 'true']
new_naama_df_false = df_naama[df_naama['correct_answer'] == 'false']

new_naama_df['label'] = new_naama_df['question'].apply(lambda x: x.split()[1])
new_naama_df_false['distractor'] = new_naama_df_false['question'].apply(lambda x: x.split()[1])

merged_df = pd.merge(new_naama_df, new_naama_df_false, on='sentence', how='inner')
new_naama_df['distractor'] = merged_df['distractor_y']
new_naama_df.drop(['correct_answer', 'gender'], axis=1, inplace=True)
df_naama = new_naama_df
df_naama


,sentence,question,label1,label2,distractor1,distractor2,dependency,animacy,match
0,The actress who was in the French director's s...,The actress hit the vase.,actress,actress,director,director,S-V,animate,True
1,The camera that was in the French director's s...,The camera hit the vase.,camera,camera,director,director,S-V,inanimate,False
2,This is the actress who this week in the Frenc...,The actress hit the vase.,actress,actress,director,director,F-G,animate,True
3,This is the camera that this week in the Frenc...,The camera hit the vase.,camera,camera,director,director,F-G,inanimate,False
4,The student who was yesterday in the class of ...,The student bored the class.,student,student,practitioner,practitioner,S-V,animate,True
...,...,...,...,...,...,...,...,...,...
75,This is the alarm that in the morning in the s...,The alarm woke up the combat soldiers.,alarm,alarm,officer,officer,F-G,inanimate,False
76,The witness who was this week in the hardened ...,The witness incriminated the accomplice.,witness,witness,criminal,criminal,S-V,animate,True
77,The evidence that was this week in the hardene...,The evidence incriminated the accomplice.,evidence,evidence,criminal,criminal,S-V,inanimate,False
78,This is the witness who this week in the harde...,The witness incriminated the accomplice.,witness,witness,criminal,criminal,F-G,animate,True


In [54]:
# add 'prompt' column

df_naama['prompt'] = df_naama['sentence'] + ' ' + df_naama['question'].apply(lambda x: ' '.join(x.split()[:1]) + ' [MASK]')+ ' ' + df_naama['question'].apply(lambda x: ' '.join(x.split()[2:]))
df_naama = df_naama[['prompt', 'label', 'distractor', 'dependency', 'animacy', 'match']]
df_naama

,prompt,label1,label2,distractor1,distractor2,dependency,animacy,match
0,The actress who was in the French director's s...,actress,actress,director,director,S-V,animate,True
1,The camera that was in the French director's s...,camera,camera,director,director,S-V,inanimate,False
2,This is the actress who this week in the Frenc...,actress,actress,director,director,F-G,animate,True
3,This is the camera that this week in the Frenc...,camera,camera,director,director,F-G,inanimate,False
4,The student who was yesterday in the class of ...,student,student,practitioner,practitioner,S-V,animate,True
...,...,...,...,...,...,...,...,...
75,This is the alarm that in the morning in the s...,alarm,alarm,officer,officer,F-G,inanimate,False
76,The witness who was this week in the hardened ...,witness,witness,criminal,criminal,S-V,animate,True
77,The evidence that was this week in the hardene...,evidence,evidence,criminal,criminal,S-V,inanimate,False
78,This is the witness who this week in the harde...,witness,witness,criminal,criminal,F-G,animate,True


prompt | label | distractor | dependency | animacy | match
- if animacy = animate then match = True, if animacy = inanimate then match = False (we can remove one of them)
- we want to aggregate results based on dependency & animacy

In [55]:
df_naama.to_csv("../data_processed/naama.csv", index=False)

In [14]:
import pandas as pd

data = {
    'Hint Sentence': [
        "Many fruits come in various colors, such as yellow.",
        "Sarah's daily routine involves going to school.",
        "One of his favorite hobbies is playing musical instruments.",
        "Cats often take a nap after a tiring day.",
        "It slipped my mind, but I forgot to complete my homework.",
        "People savor delicious food like pizza on hot summer days.",
        "During the weekends, she frequently engages in a particular activity.",
        "As he left, he had a smile on his face.",
        "Out of nowhere, a small animal suddenly dashed away.",
        "Despite searching high and low, I couldn't locate my keys."
    ],
    'prompt': [
        "The banana is [MASK].",
        "She [MASK] to school.",
        "He plays the [MASK] with his pick.",
        "The cat is [MASK].",
        "I [MASK] my homework.",
        "They are eating [MASK].",
        "She likes to [MASK].",
        "He is wearing a [MASK].",
        "The dog chased the [MASK].",
        "I can't find my [MASK]."
    ],
    'label': [
        "yellow",
        "walks",
        "guitar",
        "sleeping",
        "forgot",
        "pizza",
        "swim",
        "hat",
        "squirrel",
        "keys"
    ],
    'distractor': [
        "red",
        "swims",
        "piano",
        "jumping",
        "finished",
        "ice cream",
        "read",
        "scarf",
        "butterfly",
        "book"
    ]
}


df = pd.DataFrame(data)
df['prompt'] = df['Hint Sentence'] + ' ' + df['prompt']
df.drop(['Hint Sentence'], axis=1, inplace=True)
df = pd.concat([df, pd.DataFrame({'prompt': ['My dad loves riding his bike, and my mom loves driving her car. My [MASK] drove me to school.'], 'label': ['mom'], 'distractor': ['dad']})], ignore_index=True)
df = pd.concat([df, pd.DataFrame({'prompt': ['The city is located far from me while the park is close. It took me a long ime to get to the [MASK].'], 'label': ['city'], 'distractor': ['park']})], ignore_index=True)
df = pd.concat([df, pd.DataFrame({'prompt': ['I really like pizza and I hate beet. I usually eat [MASK].'], 'label': ['pizza'], 'distractor': ['beet']})], ignore_index=True)
df = pd.concat([df, pd.DataFrame({'prompt': ['My friend is really nice to me, while my teacher is really mean. My [MASK] screamed at me.'], 'label': ['friend'], 'distractor': ['teacher']})], ignore_index=True)
df = pd.concat([df, pd.DataFrame({'prompt': ['The dog has two wings instead of four legs, and he uses it to fly to the park.'], 'label': ['pizza'], 'distractor': ['beet']})], ignore_index=True)


new_df = df.tail(4).copy()
new_df.to_csv("../data_processed/new_test.csv", index=False)
new_df.head(20)

,prompt,label,distractor
10,"My dad loves riding his bike, and my mom loves...",mom,dad
11,The city is located far from me while the park...,city,park
12,I really like pizza and I hate beet. I usually...,pizza,beet
13,"My friend is really nice to me, while my teach...",friend,teacher
